<a href="https://colab.research.google.com/github/RuwaAbey/Finaly_Year_Project_G09/blob/main/Mutual_action_dataset_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


2.5.1+cu124
True
Tesla T4


In [3]:
!pip install mmcv==1.5.0
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.7/530.7 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 20.3 MB/s eta 0:00:00
  Created wheel for mmcv: filename=mmcv-1.5.0-py2.py3-none-any.whl size=807159 sha256=45c8b81641376451b5bd0c2a5327807a6f3bf4a3fa27ff413f3a8238998078c0
  Stored in directory: /root/.cache/pip/wheels/20/5f/a9/54775baf3e918cd5f286a769399f29d40ed5ae2a9a52811138
Successfully built mmcv
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=97cef666d6434ff4d74c482e35c76fb9ff3a5cc0ff7c4c42e01a6299b28f88e5
  Stored in directory: /root/

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/My\ Drive/G_09

/content/drive/My Drive/G_09


In [6]:
!git clone https://github.com/kennymckormick/pyskl.git

fatal: destination path 'pyskl' already exists and is not an empty directory.


In [7]:
%cd /content/drive/My\ Drive/G_09/pyskl

/content/drive/My Drive/G_09/pyskl


In [8]:
import pyskl
import numpy as np

In [47]:
import pickle

%cd /content/drive/My\ Drive/G_09

# Load the pickle file
with open('ntu60_hrnet.pkl', 'rb') as f:
    data = pickle.load(f)

# Check the top-level keys
print(data.keys())  # Should print: ['split', 'annotations']

/content/drive/My Drive/G_09
dict_keys(['split', 'annotations'])


In [10]:
annotations = data['annotations']

for annotation in annotations:
    if annotation['frame_dir'] == 'S001C001P001R001A059':
        action_58_sample = annotation
        print(annotation)

{'frame_dir': 'S001C001P001R001A059', 'label': 58, 'img_shape': (1080, 1920), 'original_shape': (1080, 1920), 'total_frames': 99, 'keypoint': array([[[[1433. ,  321. ],
         [1441. ,  312.5],
         [1429. ,  312.5],
         ...,
         [1425. ,  588.5],
         [1450. ,  676. ],
         [1425. ,  668. ]],

        [[1431. ,  323.2],
         [1439. ,  315. ],
         [1431. ,  315. ],
         ...,
         [1427. ,  588. ],
         [1448. ,  676. ],
         [1423. ,  667.5]],

        [[1433. ,  321. ],
         [1441. ,  312.5],
         [1429. ,  312.5],
         ...,
         [1425. ,  588.5],
         [1445. ,  676. ],
         [1425. ,  668. ]],

        ...,

        [[ 872. ,  323.5],
         [ 867.5,  310.8],
         [ 863.5,  310.8],
         ...,
         [ 855. ,  601.5],
         [ 859. ,  678.5],
         [ 850.5,  691.5]],

        [[ 873. ,  323.2],
         [ 868.5,  310.5],
         [ 864.5,  314.8],
         ...,
         [ 856. ,  600. ],
         [

In [11]:
action_58_sample_keypoint = action_58_sample['keypoint']
action_58_sample_keypoint_score = action_58_sample['keypoint_score']

In [21]:
from pyskl.datasets.pipelines.sampling import UniformSampleFrames

# Initialize the sampler with desired parameters
uniform_sample = UniformSampleFrames(clip_len=48, p_interval=1,num_clips=2, seed=255)

# Assuming 'results' is a dictionary containing 'total_frames', 'start_index', etc.
results = {
    'total_frames': action_58_sample['total_frames'],  # Total number of frames in the video
    'start_index': 0,  # Starting index for sampling
    'test_mode': False,  # Indicating whether it's testing or training
    'keypoint': action_58_sample['keypoint'],  # Including the 'keypoint' data
    'keypoint_score': action_58_sample['keypoint_score'],  # Including the 'keypoint_score' data
    'img_shape': (1080, 1920)
}

# Call the sampler to sample the frames
processed_results = uniform_sample(results)

# The processed results will contain the 'frame_inds' with the sampled frames
print(processed_results['frame_inds'].shape)

(96,)


In [22]:
from pyskl.datasets.pipelines.pose_related import PoseDecode
# Create an instance of PoseDecode
pose_decode = PoseDecode()

# Apply PoseDecode on the results
processed_results = pose_decode(processed_results)

# Get the processed keypoints and keypoint scores for the selected frames
selected_keypoints = processed_results['keypoint']  # Extracted keypoints for sampled frames
selected_scores = processed_results['keypoint_score']  # Extracted keypoint scores for sampled frames

# Print the output
print(selected_keypoints.shape)  # (n_frames, 25, 2)
print(selected_scores.shape)     # (n_frames, 25)

(2, 96, 17, 2)
(2, 96, 17)


In [24]:
import numpy as np

coordinates = action_58_sample_keypoint
confidence_scores = action_58_sample_keypoint_score

# Assuming these are your original arrays
coords = selected_keypoints  # Replace with actual data
conf = selected_scores      # Replace with actual data

# Transpose coordinates to (M, C, T, V)
coords = np.transpose(coords, (0, 3, 1, 2))  # Shape: (2, 2, 99, 17)

# Expand confidence scores to match dimensions (M, 1, T, V)
conf = np.expand_dims(conf, axis=1)  # Shape: (2, 1, 99, 17)

# Concatenate along the second axis (C)
combined = np.concatenate((coords, conf), axis=1)  # Shape: (2, 3, 99, 17)

# Swap M and last dimension (M should be last)
combined = np.transpose(combined, (1, 2, 3, 0))  # Shape: (3, 99, 17, 2)

# Add batch dimension (N=1)
final_array = np.expand_dims(combined, axis=0)  # Shape: (1, 3, 99, 17, 2)

print(final_array.shape)  # Expected Output: (1, 3, 99, 17, 2)
print(final_array)

x = final_array
x.shape

(1, 3, 96, 17, 2)
[[[[[1.4330000e+03 4.7125000e+02]
    [1.4410000e+03 4.7125000e+02]
    [1.4290000e+03 4.5825000e+02]
    ...
    [1.4250000e+03 4.6675000e+02]
    [1.4500000e+03 4.8825000e+02]
    [1.4250000e+03 4.7125000e+02]]

   [[1.4330000e+03 4.6925000e+02]
    [1.4410000e+03 4.7350000e+02]
    [1.4290000e+03 4.5625000e+02]
    ...
    [1.4250000e+03 4.6475000e+02]
    [1.4450000e+03 4.9050000e+02]
    [1.4250000e+03 4.7350000e+02]]

   [[1.4330000e+03 4.6925000e+02]
    [1.4410000e+03 4.7350000e+02]
    [1.4290000e+03 4.5625000e+02]
    ...
    [1.4250000e+03 4.6475000e+02]
    [1.4450000e+03 4.9050000e+02]
    [1.4250000e+03 4.7350000e+02]]

   ...

   [[8.6950000e+02 1.0095000e+03]
    [8.6550000e+02 1.0185000e+03]
    [8.5700000e+02 1.0140000e+03]
    ...
    [8.5700000e+02 1.0450000e+03]
    [8.5700000e+02 1.0500000e+03]
    [8.5250000e+02 1.0540000e+03]]

   [[8.6850000e+02 1.0080000e+03]
    [8.6850000e+02 1.0170000e+03]
    [8.6000000e+02 1.0125000e+03]
    ...
    [8.5

(1, 3, 96, 17, 2)

In [40]:
import numpy as np
import numpy as geek  # (Do you really need this? 'geek' is another alias for numpy)
from pyskl.datasets.pipelines.sampling import UniformSampleFrames
from pyskl.datasets.pipelines.pose_related import PoseDecode


all_videos_data = []

# Loop through each annotation in data['annotations']
for i, annotation in enumerate(data['annotations']):

    if i >= 30:  # Stop after processing 30 videos
        break

    # Initialize the sampler with desired parameters
    uniform_sample = UniformSampleFrames(clip_len=48, p_interval=1,num_clips=2, seed=255)

    # Assuming 'results' is a dictionary containing 'total_frames', 'start_index', etc.
    results = {
    'total_frames': annotation['total_frames'],  # Total number of frames in the video
    'start_index': 0,  # Starting index for sampling
    'test_mode': False,  # Indicating whether it's testing or training
    'keypoint': annotation['keypoint'],  # Including the 'keypoint' data
    'keypoint_score': annotation['keypoint_score'],  # Including the 'keypoint_score' data
    'img_shape': (1080, 1920)
      }

    # Call the sampler to sample the frames
    processed_results = uniform_sample(results)

    pose_decode = PoseDecode()

    # Apply PoseDecode on the results
    processed_results = pose_decode(processed_results)

    # Get the processed keypoints and keypoint scores for the selected frames
    #selected_keypoints = processed_results['keypoint']  # Extracted keypoints for sampled frames
    #selected_scores = processed_results['keypoint_score']  # Extracted keypoint scores for sampled frames

    coordinates = processed_results['keypoint']
    confidence_scores = processed_results['keypoint_score']

    # Assuming these are your original arrays
    coords = coordinates  # Replace with actual data
    conf = confidence_scores  # Replace with actual data



    # Transpose coordinates to (M, C, T, V)
    coords = np.transpose(coords, (0, 3, 1, 2))  # Shape: (2, 2, 99, 17)

    # Expand confidence scores to match dimensions (M, 1, T, V)
    conf = np.expand_dims(conf, axis=1)  # Shape: (2, 1, 99, 17)

    # Concatenate along the second axis (C)
    combined = np.concatenate((coords, conf), axis=1)  # Shape: (2, 3, 99, 17)

    # Swap M and last dimension (M should be last)
    combined = np.transpose(combined, (1, 2, 3, 0))  # Shape: (3, 99, 17, 2)

    # Add batch dimension (N=1)
    final_array = combined # Shape: (1, 3, 99, 17, 2)

    all_videos_data.append(final_array)

print(len(all_videos_data))

#all_videos_data_np = np.array(all_videos_data)  # Convert list to NumPy array
print(all_videos_data[29].shape)  # Check the shape of the array
all_videos_data = np.array(all_videos_data)


30
(3, 96, 17, 1)


In [41]:
all_videos_data.shape

(30, 3, 96, 17, 1)

In [42]:
all_videos_data[0]

array([[[[1.0320000e+03],
         [1.0410000e+03],
         [1.0235000e+03],
         ...,
         [1.0280000e+03],
         [1.0630000e+03],
         [1.0370000e+03]],

        [[1.0320000e+03],
         [1.0410000e+03],
         [1.0230000e+03],
         ...,
         [1.0270000e+03],
         [1.0630000e+03],
         [1.0360000e+03]],

        [[1.0320000e+03],
         [1.0410000e+03],
         [1.0235000e+03],
         ...,
         [1.0280000e+03],
         [1.0630000e+03],
         [1.0370000e+03]],

        ...,

        [[1.0390000e+03],
         [1.0480000e+03],
         [1.0350000e+03],
         ...,
         [1.0300000e+03],
         [1.0660000e+03],
         [1.0390000e+03]],

        [[1.0380000e+03],
         [1.0470000e+03],
         [1.0340000e+03],
         ...,
         [1.0290000e+03],
         [1.0650000e+03],
         [1.0380000e+03]],

        [[1.0370000e+03],
         [1.0500000e+03],
         [1.0330000e+03],
         ...,
         [1.0280000e+03],
        

In [84]:
import numpy as np
from pyskl.datasets.pipelines.sampling import UniformSampleFrames
from pyskl.datasets.pipelines.pose_related import PoseDecode

allowed_actions = {
    "A050", "A051", "A052", "A053", "A054", "A055", "A056", "A057",
    "A058", "A059", "A060", "A106", "A107", "A108", "A109", "A110",
    "A111", "A112", "A113", "A114", "A115", "A116", "A117", "A118",
    "A119", "A120"
}

all_videos_data = []

# Loop through each annotation in data['annotations']
for annotation in data['annotations']:
    frame_dir = annotation['frame_dir']
    action_code = frame_dir[-4:]
    #print(action_code)

    if action_code not in allowed_actions:
        continue

    #print(annotation['frame_dir'] )

    # Initialize the sampler with desired parameters
    uniform_sample = UniformSampleFrames(clip_len=48, p_interval=1, num_clips=2, seed=255)

    # Define the results dictionary
    results = {
        'total_frames': annotation['total_frames'],
        'start_index': 0,
        'test_mode': False,
        'keypoint': annotation['keypoint'],
        'keypoint_score': annotation['keypoint_score'],
        'img_shape': (1080, 1920)
    }

    # Sample frames
    processed_results = uniform_sample(results)

    # Apply PoseDecode
    pose_decode = PoseDecode()
    processed_results = pose_decode(processed_results)

    # Extract keypoints and confidence scores
    coords = processed_results['keypoint']
    conf = processed_results['keypoint_score']



    # Transpose coordinates to (M, C, T, V)
    coords = np.transpose(coords, (0, 3, 1, 2))  # Shape: (2, 2, 99, 17)

    # Expand confidence scores to match dimensions (M, 1, T, V)
    conf = np.expand_dims(conf, axis=1)  # Shape: (2, 1, 99, 17)

    # Concatenate along the second axis (C)
    combined = np.concatenate((coords, conf), axis=1)  # Shape: (2, 3, 99, 17)

    # Swap M and last dimension (M should be last)
    combined = np.transpose(combined, (1, 2, 3, 0))  # Shape: (3, 99, 17, 2)

    # Add batch dimension (N=1)
    final_array = combined # Shape: (1, 3, 99, 17, 2)

    all_videos_data.append(final_array)

     # Ensure fixed shape (T=96)
    if len(all_videos_data) == 2100:
        break  # Skip samples with different frame counts

    #all_videos_data.append(combined)  # Append without extra batch dim

all_videos_data = np.array(all_videos_data)
print(all_videos_data.shape)
type(all_videos_data)

(2100, 3, 96, 17, 2)


numpy.ndarray

In [87]:
import os
import numpy as np

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the path
folder_path = "/content/drive/My Drive/zoom_transformer/data"
file_path = os.path.join(folder_path, "NTU_mutual_actions_1000_videos.npy")

# Create the directory if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Save the NumPy array
np.save(file_path, all_videos_data)

print(f"File saved at: {file_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
File saved at: /content/drive/My Drive/zoom_transformer/data/NTU_mutual_actions_1000_videos.npy


In [88]:
# Load the saved NumPy array
loaded_data = np.load("/content/drive/My Drive/zoom_transformer/data/NTU_mutual_actions_1000_videos.npy")

# Check the shape of the loaded data
print(loaded_data.shape)  # Should be (2100, 3, 96, 17, 2)


(2100, 3, 96, 17, 2)


In [92]:
import numpy as np
import pickle
from pyskl.datasets.pipelines.sampling import UniformSampleFrames
from pyskl.datasets.pipelines.pose_related import PoseDecode

# Define allowed actions
allowed_actions = {
    "A050", "A051", "A052", "A053", "A054", "A055", "A056", "A057",
    "A058", "A059", "A060", "A106", "A107", "A108", "A109", "A110",
    "A111", "A112", "A113", "A114", "A115", "A116", "A117", "A118",
    "A119", "A120"
}

sample_names = []  # List to store sample names
labels = []        # List to store corresponding labels

all_videos_data = []  # List to store the processed data (keypoints, confidence scores, etc.)

# Loop through each annotation in data['annotations']
for i, annotation in enumerate(data['annotations']):
    # Get the action class code from the frame directory (or another field)
    frame_dir = annotation['frame_dir']
    action_code = frame_dir[-4:]  # Assuming action code is the last 4 characters in frame_dir

    # Skip the samples that are not in the allowed actions
    if action_code not in allowed_actions:
        continue

    # Generate sample name dynamically
    sample_name = f"sample_{i+1}"

    # Extract the label for this sample
    label = annotation['label']  # Assuming 'label' field exists in annotations

    # Append to the lists
    sample_names.append(sample_name)
    labels.append(label)

    # Stop at 2100 samples
    if len(all_videos_data) == 2100:
        break

# Define the path for saving the pickle file
pickle_filename = '/content/drive/My Drive/zoom_transformer/data/annotation_label.pkl'

# Save the sample names and labels to the pickle file
with open(pickle_filename, 'wb') as f:
    pickle.dump((sample_names, labels), f)

print(f"Saved the pickle file with sample names and labels at: {pickle_filename}")


Saved the pickle file with sample names and labels at: /content/drive/My Drive/zoom_transformer/data/annotation_label.pkl
